<a href="https://colab.research.google.com/github/jgillet4/hackWestern7/blob/master/NLP_MachineLearning_StoryBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NLP Processing with N-gram, Model 1

In [3]:
# >>> import nltk
# >>> nltk.download('reuters')
# >>> nltk.download('punkt')

from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import Counter, defaultdict

# Create a placeholder for model
model = defaultdict(lambda: defaultdict(lambda: 0))

# Count frequency of co-occurance  
for sentence in reuters.sents():
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        model[(w1, w2)][w3] += 1
 
# Let's transform the counts to probabilities
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

print("hello")

hello


In [4]:
#use the above model to predict for two words, the next word
dict(model["today","the"])

{'Bank': 0.05555555555555555,
 'European': 0.05555555555555555,
 'Higher': 0.05555555555555555,
 'Italian': 0.05555555555555555,
 'Turkish': 0.05555555555555555,
 'company': 0.16666666666666666,
 'emirate': 0.05555555555555555,
 'increase': 0.05555555555555555,
 'newspaper': 0.05555555555555555,
 'options': 0.05555555555555555,
 'overseas': 0.05555555555555555,
 'pound': 0.05555555555555555,
 'price': 0.1111111111111111,
 'public': 0.05555555555555555,
 'time': 0.05555555555555555}

# Creating full sentences

In [ ]:
import random

# starting words
text = ["today", "the"]
sentence_finished = False
 
while not sentence_finished:
  # select a random probability threshold  
  r = random.random()
  accumulator = .0

  for word in model[tuple(text[-2:])].keys():
      accumulator += model[tuple(text[-2:])][word]
      # select words that are above the probability threshold
      if accumulator >= r:
          text.append(word)
          break

  if text[-2:] == [None, None]:
      sentence_finished = True
 
print (' '.join([t for t in text if t]))


In [14]:
!git clone https://github.com/jgillet4/hackWestern7.git


fatal: destination path 'hackWestern7' already exists and is not an empty directory.


In [20]:
!cd hackWestern7/

In [22]:
!ls


hackWestern7  sample_data


In [23]:
%cd hackWestern7/

/content/hackWestern7


In [32]:
!ls
!git status
!git pull
!git add .
!git commit -m "Machine learning nlp tutorials"
!git config --global user.email "tjamesch@uwo.ca"
!git config --global user.name "tonigogitter"
!git push

hello.go
On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean
Already up to date.
On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address
